# Pipeline for Cyberbullying Text Analysis With Gemini API

This notebook demonstrate the integration of Gemini API into an NLP pipeline for cyberbullying detecttion and analysis, using datasets like HateSpeech and OLID.

The workflow covers the following steps:
- Data Preprocessing
- Data Splitting, Balancing and Unifying Datasets into a Common Format
- Gemini API Integration
- Fine-tuning
- Feature extraction
- visualization and interpretability
- Comparisons and Analysis of different methods and aproaches (TF-IDF, BERT, RoBERTa...) with Gemini's capabilities in the context of detecting toxic or hate speech content.

In [69]:
!pip install -q google-generativeai

In [2]:
# !pip install google-genai
!pip install python-dotenv

In [66]:
from dotenv import dotenv_values
import os
import pandas as pd

config = dotenv_values(".env")

In [72]:
# from google import genai
import google.generativeai as genai
api_key=config["GEMINI_API_KEY"]

# client = genai.Client(api_key=api_key)

In [74]:
genai.configure(api_key=api_key)

In [7]:
for model_info in client.models.list():
    print(model_info.name)

In [75]:
for i, m in zip(range(5), genai.list_tuned_models()):
    print(m.name)

In [76]:
base_model = [
    m for m in genai.list_models()
    if "createTunedModel" in m.supported_generation_methods and
       "flash" in m.name][0]
base_model# model = "gemini-2.5-pro-exp-03-25"

In [78]:
import random

name = f'generate-num-{random.randint(0,10000)}'
operation = genai.create_tuned_model(
    # You can use a tuned model here too. Set `source_model="tunedModels/..."`
    source_model=base_model.name,
    training_data=[
        {
            'text_input': '1',
            'output': '2',
        },{
            'text_input': '3',
            'output': '4',
        },{
            'text_input': '-3',
            'output': '-2',
        },{
            'text_input': 'twenty two',
            'output': 'twenty three',
        },{
            'text_input': 'two hundred',
            'output': 'two hundred one',
        },{
            'text_input': 'ninety nine',
            'output': 'one hundred',
        },{
            'text_input': '8',
            'output': '9',
        },{
            'text_input': '-98',
            'output': '-97',
        },{
            'text_input': '1,000',
            'output': '1,001',
        },{
            'text_input': '10,100,000',
            'output': '10,100,001',
        },{
            'text_input': 'thirteen',
            'output': 'fourteen',
        },{
            'text_input': 'eighty',
            'output': 'eighty one',
        },{
            'text_input': 'one',
            'output': 'two',
        },{
            'text_input': 'three',
            'output': 'four',
        },{
            'text_input': 'seven',
            'output': 'eight',
        }
    ],
    id = name,
    epoch_count = 100,
    batch_size=4,
    learning_rate=0.001,
)

In [79]:
model = genai.get_tuned_model(f'tunedModels/{name}')

model

In [86]:
model.state

In [84]:
operation.metadata

In [85]:
import time

for status in operation.wait_bar():
    time.sleep(30)

In [ ]:
# operation.cancel()

In [ ]:
import pandas as pd
import seaborn as sns

model = operation.result()

snapshots = pd.DataFrame(model.tuning_task.snapshots)

sns.lineplot(data=snapshots, x = 'epoch', y='mean_loss')


## Fine-Tuning Gemini on Toxic Language Data

We would like to conduct 2 eperiments:
1. We provide the texts with good and bad labels and fine-tune the model for classification of hate speech.
2. We provide the texts with their labels, and additionally we provide the n-grams and its TF-IDF scores and fine-tune the model for classification of hate speech.

Fine-tuning allows adapting Gemini to specific tasks like cyberbullying detection. Google’s API supports supervised fine-tuning by providing a custom dataset of input-output pairs.

https://ai.google.dev/gemini-api/docs/model-tuning#:~:text=Format

https://ai.google.dev/gemini-api/docs/model-tuning/tutorial?lang=python#:~:text=training_dataset%3Dtypes.TuningDataset%28%20examples%3D,tune

Gemini fine-tuning currently requires prompt-response style data (single-turn examples);

For a classification task (toxic vs not toxic), a common approach is to supply the toxic text as the input and a desired label or outcome as the output. For example, the training set might consist of entries like ```{"text_input": "You are an idiot", "output": "toxic"}``` and ```{"text_input": "I love puppies", "output": "not toxic"}```.

In [13]:
## Data preparation

In [14]:
data = pd.read_csv("data/balanced-merged-data.csv")
good_ngrams_df = pd.read_csv("data/top_good_ngrams.csv")
bad_ngrams_df = pd.read_csv("data/top_bad_ngrams.csv")

In [19]:
good_ngrams_df.head()

In [20]:
good_ngrams_df.rename(columns={"N-gram": "ngram", "Score":"score"}, inplace=True)

In [21]:
bad_ngrams_df.head()

In [22]:
bad_ngrams_df.rename(columns={"N-gram": "ngram", "Score":"score"}, inplace=True)

In [15]:
from google.genai import types

In [40]:
train_samples = []

# pick 5 rows from the dataframe
dat = data.sample(n=5)
for index, row in data.iterrows():
    train_samples.append(types.TuningExample(
            text_input=row["text"],
            output=row["label"],
            # good_ngrams=good_ngrams_df["ngram"].tolist(),
            # good_ngrams_scores=good_ngrams_df["score"].tolist(),
            # bad_ngrams=bad_ngrams_df["ngram"].tolist(),
            # bad_ngrams_scores=bad_ngrams_df["score"].tolist(),
        ))
        
        # ngrams_weights=good_ngrams_df["weight"].tolist()
    
training_dataset = types.TuningDataset(examples=train_samples)

In [41]:
train_samples

In [42]:
train_dataset = types.TuningDataset(examples=train_samples)

In [37]:
# TODO: how to use validation set
# TODO: how to use test set
# TODO: how to use metrics


In [58]:
model = "models/gemini-1.5-flash-001-tuning"
# run the fine-tuning jog
tuning_job = client.tunings.tune(
    base_model='models/gemini-1.5-flash-001-tuning',
    training_dataset=training_dataset,
    config=types.CreateTuningJobConfig(
        epoch_count= 5,
        batch_size=4,
        learning_rate=0.001,
        tuned_model_display_name="test tuned model"
    )
)


In [61]:
tuning_job

In [59]:
print(f"Training job: {train_job.name} ")


In [62]:
print(train_job.state)




In [63]:
metrics = train_job.training_metrics
print(metrics)


In [54]:
tuned_model = client.get_model(name="test tuned model")


In [56]:
# generate content with the tuned model
response = client.models.generate_content(
    model=train_job.tuned_model.model,
    contents='III',
)

print(response.text)

In [51]:
model_name = train_job.tuned_model.model# your custom model ID
response = client.models.generate_content(
    model=model_name,
    contents="Why don't you just shut up and leave me alone?"
)
print(response.text)


In [64]:
# create tuning model
training_dataset =  [
    ["1", "2"],
    ["3", "4"],
    ["-3", "-2"],
    ["twenty two", "twenty three"],
    ["two hundred", "two hundred one"],
    ["ninety nine", "one hundred"],
    ["8", "9"],
    ["-98", "-97"],
    ["1,000", "1,001"],
    ["10,100,000", "10,100,001"],
    ["thirteen", "fourteen"],
    ["eighty", "eighty one"],
    ["one", "two"],
    ["three", "four"],
    ["seven", "eight"],
]
training_dataset=types.TuningDataset(
    examples=[
        types.TuningExample(
            text_input=i,
            output=o,
        )
        for i,o in training_dataset
    ],
)
tuning_job = client.tunings.tune(
    base_model='models/gemini-1.5-flash-001-tuning',
    training_dataset=training_dataset,
    config=types.CreateTuningJobConfig(
        epoch_count= 5,
        batch_size=4,
        learning_rate=0.001,
        tuned_model_display_name="test tuned model"
    )
)

# generate content with the tuned model
response = client.models.generate_content(
    model=tuning_job.tuned_model.model,
    contents='III',
)

print(response.text)